Setup dependencies and logging

In [ ]:
pip install -q -r requirements.txt

In [1]:
import logging
import json
from IPython.display import Pretty, HTML
import urllib3

from pyfhirclient.FhirClient import FhirClient

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

# Disable the unverified HTTPS request warnings
urllib3.disable_warnings()

In [2]:
def setup_fhir_client() -> FhirClient:
    fhir_client = FhirClient(
        base_url='https://connectathon.epic.com/Interconnect-Fhir-Unsecure/api/FHIR/R4/',
        verify_ssl=False,
        extra_headers={
            'Epic-Client-ID': 'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00'
        })

    return fhir_client

In [3]:
client = setup_fhir_client()

# TODO: No results
search_results = client.search('Patient', {
    # 'family': 'Smith',
    'gender': 'male',
    # 'birthdate': '1965-09-01',
    # 'address-state': 'NC',
})
Pretty(json.dumps(search_results, indent=4))

{
    "resourceType": "Bundle",
    "type": "searchset",
    "total": 0,
    "link": [
        {
            "relation": "self",
            "url": "https://connectathon.epic.com/Interconnect-FHIR-OAuth/api/FHIR/R4/Patient?gender=male"
        }
    ],
    "entry": [
        {
            "fullUrl": "urn:uuid:eab41186-8f42-4fc0-a44b-989e90b2dee1",
            "resource": {
                "resourceType": "OperationOutcome",
                "issue": [
                    {
                        "severity": "warning",
                        "code": "processing",
                        "details": {
                            "coding": [
                                {
                                    "system": "urn:oid:1.2.840.114350.1.13.1.1.7.2.657369",
                                    "code": "4101",
                                    "display": "Resource request returns no results."
                                }
                            ],
                        

In [4]:
patient = search_results['entry'][0]
Pretty(json.dumps(patient, indent=4))

{
    "fullUrl": "urn:uuid:eab41186-8f42-4fc0-a44b-989e90b2dee1",
    "resource": {
        "resourceType": "OperationOutcome",
        "issue": [
            {
                "severity": "warning",
                "code": "processing",
                "details": {
                    "coding": [
                        {
                            "system": "urn:oid:1.2.840.114350.1.13.1.1.7.2.657369",
                            "code": "4101",
                            "display": "Resource request returns no results."
                        }
                    ],
                    "text": "Resource request returns no results."
                }
            }
        ]
    },
    "search": {
        "mode": "outcome"
    }
}

In [5]:
HTML(patient['resource']['text']['div'])

KeyError: 'text'

In [ ]:
everything = client.everything('Patient', patient['resource']['id'], count=1000)
Pretty(json.dumps(everything, indent=4))

In [ ]:
medication_administration_results = client.search('MedicationAdministration', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(medication_administration_results, indent=4))

In [ ]:
observation_search_results = client.search('Observation', query_params={
    'patient': patient['resource']['id']
})

Pretty(json.dumps(observation_search_results, indent=4))        

In [ ]:
encounter_search_results = client.search('Encounter', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(encounter_search_results, indent=4))

In [ ]:
client.close()